
# Store and Query Portfolio Data with SQL

This notebook demonstrates how to store:
- Monte Carlo simulated portfolio losses
- Predicted PD, LGD, EAD values
- Summary metrics

in a SQL database (SQLite for local demonstration) and perform basic queries.


In [2]:
# Import necessary libraries
import sqlite3
import pandas as pd
from pathlib import Path

# Base project folder
project_path = Path("/home/skumar/Desktop/credit-risk-analytics")

# Paths for data and SQL
data_path = project_path / "data/input_raw/credit_portfolio.csv"
monte_carlo_path = project_path / "data/processed/monte_carlo_results.csv"
sql_path = project_path / "sql"
sql_path.mkdir(parents=True, exist_ok=True)  # make sure folder exists

# Path to SQLite database
db_path = sql_path / "credit_portfolio.db"

print("✅ Paths set. Database will be at:", db_path)


✅ Paths set. Database will be at: /home/skumar/Desktop/credit-risk-analytics/sql/credit_portfolio.db


## Load Portfolio Data and Monte Carlo Results


In [3]:
# Load portfolio CSV
portfolio_df = pd.read_csv(data_path)

# For demonstration, assume PD, LGD, EAD predictions are present
portfolio_df["PD_pred"] = portfolio_df["PD"]
portfolio_df["LGD_pred"] = portfolio_df["LGD"]
portfolio_df["EAD_pred"] = portfolio_df["EAD"]

# Load or simulate Monte Carlo results
try:
    mc_results = pd.read_csv(monte_carlo_path)
except FileNotFoundError:
    import numpy as np
    n_simulations = 10000
    portfolio_losses = []

    for i in range(n_simulations):
        random_numbers = np.random.rand(len(portfolio_df))
        defaults = random_numbers < portfolio_df["PD_pred"]
        losses = defaults * portfolio_df["LGD_pred"] * portfolio_df["EAD_pred"]
        portfolio_losses.append(losses.sum())
    mc_results = pd.DataFrame({"portfolio_loss": portfolio_losses})
    mc_results.to_csv(monte_carlo_path, index=False)
    print("✅ Monte Carlo results simulated and saved")


✅ Monte Carlo results simulated and saved


## Create Database and Tables


In [4]:
# Connect to SQLite (creates file if it doesn't exist)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create portfolio table
cursor.execute("""
CREATE TABLE IF NOT EXISTS portfolio (
    id INTEGER PRIMARY KEY,
    exposure REAL,
    PD REAL,
    LGD REAL,
    EAD REAL,
    PD_pred REAL,
    LGD_pred REAL,
    EAD_pred REAL
)
""")

# Create simulation results table
cursor.execute("""
CREATE TABLE IF NOT EXISTS simulation_results (
    id INTEGER PRIMARY KEY,
    simulation_num INTEGER,
    portfolio_loss REAL
)
""")

conn.commit()
print("✅ Database and tables created")


✅ Database and tables created


## Insert Portfolio Data and Simulation Results


In [7]:
# Ensure predicted columns exist
portfolio_df["PD_pred"] = portfolio_df["PD"]
portfolio_df["LGD_pred"] = portfolio_df["LGD"]
portfolio_df["EAD_pred"] = portfolio_df["EAD"]

# Only use columns that actually exist in your DataFrame
columns_to_insert = ['PD','LGD','EAD','PD_pred','LGD_pred','EAD_pred']

# Convert to records for SQLite insertion
portfolio_records = portfolio_df[columns_to_insert].to_records(index=False)

# Insert into database
cursor.executemany("""
INSERT INTO portfolio (PD, LGD, EAD, PD_pred, LGD_pred, EAD_pred)
VALUES (?, ?, ?, ?, ?, ?)
""", portfolio_records)

conn.commit()
print("✅ Portfolio data inserted successfully")


✅ Portfolio data inserted successfully


In [8]:
simulation_records = [(i+1, loss) for i, loss in enumerate(mc_results['portfolio_loss'])]
cursor.executemany("""
INSERT INTO simulation_results (simulation_num, portfolio_loss)
VALUES (?, ?)
""", simulation_records)

conn.commit()
print("✅ Simulation results inserted")


✅ Simulation results inserted


## Query Examples


In [9]:
query = "SELECT * FROM simulation_results ORDER BY portfolio_loss DESC LIMIT 5"
top_losses = pd.read_sql_query(query, conn)
top_losses


,id,simulation_num,portfolio_loss
0,450,450,9.111868e+06
1,6944,6944,8.940465e+06
2,8427,8427,8.725930e+06
3,7921,7921,8.572688e+06
4,4423,4423,8.428916e+06


In [10]:
query = """
SELECT 
    AVG(portfolio_loss) AS expected_loss,
    MAX(portfolio_loss) AS max_loss,
    MIN(portfolio_loss) AS min_loss
FROM simulation_results
"""
summary_stats = pd.read_sql_query(query, conn)
summary_stats


,expected_loss,max_loss,min_loss
0,5.675572e+06,9.111868e+06,3.105741e+06


In [11]:
query = """
SELECT 
    AVG(portfolio_loss) AS expected_loss,
    MAX(portfolio_loss) AS max_loss,
    MIN(portfolio_loss) AS min_loss
FROM simulation_results
"""
summary_stats = pd.read_sql_query(query, conn)
summary_stats


,expected_loss,max_loss,min_loss
0,5.675572e+06,9.111868e+06,3.105741e+06


## Close Database Connection


In [12]:
conn.close()
print("✅ Database connection closed")


✅ Database connection closed
